**(2021-01-20 WIC:)** This is Peter Yoachim's AstrometryMetrics.ipynb with a couple of very minor tweaks to run on DataLab and sciserver.

This notebook assumes you are using sims_maf version >= 1.1, and have 'setup sims_maf' in your shell. 

## Demonstrate the proper motion and parallax metrics. ##

For both of these metrics, visits in all bands are used (unless restricted by the user via a sqlconstraint). 
The astrometric error for each visit, for a star of the specified brightness and spectral type, is estimated from the magnitude of the star and the m5 limit of each visit (recorded by opsim). 
The parallax error is estimated by assuming that the proper motion is perfectly fit (or zero); the proper motion error is likewise estimated by assuming the parallax is perfectly fit (or zero). 
In both cases, the effects of refraction is not currently included in the astrometric error. 

Here is a link to the [code](https://github.com/lsst/sims_maf/blob/master/python/lsst/sims/maf/metrics/calibrationMetrics.py) for the proper motion and parallax metrics. 
The 'ra_pi_amp' and 'dec_pi_amp' columns referred to in the parallax metric are stacker columns which calculate the parallax amplitude (in ra and dec) for each visit, using this [code](https://github.com/lsst/sims_maf/blob/master/python/lsst/sims/maf/stackers/generalStackers.py). 

In [ ]:
# 2021-01-20 - try running this on DataLab. 

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metricBundles as metricBundles

In [3]:
# root location for opsim. Uncomment as appropriate depending on whether you are on
# sciserver or datalab

# On DataLab (notice subdirectories for each category):
dbroot = '/sims_maf/fbs_1.5/baseline/'

# On Sciserver
# dbroot = '/home/idies/workspace/lsst_cadence/FBS_1.5/

In [4]:
# Set up the database connection
# Database available from: http://astro-lsst-01.astro.washington.edu:8081/db_gzip/baseline2018a.db.gz
opsdb = db.OpsimDatabase(dbroot+'baseline_v1.5_10yrs.db')
outDir = 'astrometry_test'
resultsDb = db.ResultsDb(outDir=outDir)

In [5]:
# (Use tab in this cell to do auto-complete to learn what's available)
# metrics.ParallaxDcrDegenMetric

Define our basic sqlconstraint (we'll only look at the first three years here) and define the metrics and slicer. 

In [10]:
sql = 'night < %i' % (365.25*3) # See How well we do after year 3
slicer = slicers.HealpixSlicer(nside=64)
metricList = []
metricList.append(metrics.ParallaxMetric())
metricList.append(metrics.ParallaxMetric(metricName='parallax normed', normalize=True))
metricList.append(metrics.ParallaxDcrDegenMetric())
metricList.append(metrics.ProperMotionMetric())
metricList.append(metrics.ProperMotionMetric(metricName='properMotion normed', normalize=True))

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes


Add a summary metric to compute for each of the metricBundles. 

In [11]:
summaryList = [metrics.MedianMetric()]

In [12]:
bundleList = []
plotDict = {'percentileClip': 95.}
for metric in metricList:
    bundleList.append(metricBundles.MetricBundle(metric,slicer,sql, summaryMetrics=summaryList,
                                                plotDict=plotDict))

In [ ]:
bundleDict = metricBundles.makeBundlesDictFromList(bundleList)
bgroup = metricBundles.MetricBundleGroup(bundleDict, opsdb, outDir=outDir, resultsDb=resultsDb)
bgroup.runAll()

Querying database SummaryAllProps with constraint night < 1095 for columns ['fiveSigmaDepth', 'observationStartMJD', 'fieldRA', 'observationStartLST', 'filter', 'altitude', 'seeingFwhmGeom', 'fieldDec']
Found 657138 visits
Running:  ['opsim_parallax_night_lt_1095_HEAL', 'opsim_parallax_normed_night_lt_1095_HEAL', 'opsim_ParallaxDcrDegenMetric_night_lt_1095_HEAL', 'opsim_properMotion_night_lt_1095_HEAL', 'opsim_properMotion_normed_night_lt_1095_HEAL']


/data0/lsst_stack/stack/miniconda3-py37_4.8.2-cb4e2dc/Linux64/sims_maf/2.13.0.sims-61-g3ee711b3+fffdf3dce1/python/lsst/sims/maf/metrics/calibrationMetrics.py:85: RuntimeWarning: divide by zero encountered in true_divide
  sigma_B = position_errors/dec_pi_amp
/data0/lsst_stack/stack/miniconda3-py37_4.8.2-cb4e2dc/Linux64/sims_maf/2.13.0.sims-61-g3ee711b3+fffdf3dce1/python/lsst/sims/maf/metrics/calibrationMetrics.py:87: RuntimeWarning: divide by zero encountered in double_scalars
  sigma_dec = np.sqrt(1./np.sum(1./sigma_B**2))


In [ ]:
bgroup.plotAll(closefigs=False)

By default, the proper motion and parallax metrics assume a flat SED with an r-band magnitude of 20, however this can be specified by the user when setting up the metric.

In [10]:
rmags = {'faint':25, 'bright':18}
specTypes = ['B', 'K']
metricList = []
plotDict = {'percentileClip': 95.}
for mag in rmags:
    for specType in specTypes:
        metricList.append(metrics.ParallaxMetric(rmag=rmags[mag], SedTemplate=specType, 
                                                 metricName='parallax_'+mag+'_'+specType))
        metricList.append(metrics.ProperMotionMetric(rmag=rmags[mag], SedTemplate=specType, 
                                                     metricName='properMotion'+mag+'_'+specType))

In [11]:
bundlesSpec = []
for metric in metricList:
    bundlesSpec.append(metricBundles.MetricBundle(metric,slicer,sql, summaryMetrics=summaryList,
                                                plotDict=plotDict))
bundlesSpec = metricBundles.makeBundlesDictFromList(bundlesSpec)

In [12]:
bgroup = metricBundles.MetricBundleGroup(bundlesSpec, opsdb, outDir=outDir, resultsDb=resultsDb)
bgroup.runAll()

Querying database SummaryAllProps with constraint night < 1095 for columns ['filter', 'fiveSigmaDepth', 'observationStartMJD', 'fieldDec', 'seeingFwhmGeom', 'fieldRA']
Found 696705 visits
Running:  ['opsim_parallax_faint_B_night_lt_1095_HEAL', 'opsim_properMotionfaint_B_night_lt_1095_HEAL', 'opsim_parallax_faint_K_night_lt_1095_HEAL', 'opsim_properMotionfaint_K_night_lt_1095_HEAL', 'opsim_parallax_bright_B_night_lt_1095_HEAL', 'opsim_properMotionbright_B_night_lt_1095_HEAL', 'opsim_parallax_bright_K_night_lt_1095_HEAL', 'opsim_properMotionbright_K_night_lt_1095_HEAL']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


In [ ]:
bgroup.plotAll(closefigs=False)

In [14]:
print('Flat SED:')
for bundle in bundleList:
    print(bundle.metric.name, bundle.summaryValues)
print('B and K stars:')
for bundle in bundlesSpec:
    print(bundlesSpec[bundle].metric.name, bundlesSpec[bundle].summaryValues)

Flat SED:
parallax {'Median': 1.1205172953865949}
parallax Normed {'Median': 0.73526281038490993}
properMotion {'Median': 0.93208904119636315}
properMotion Normed {'Median': 0.17288143229954442}
B and K stars:
parallax_faint_B {'Median': 27.929436281538997}
properMotionfaint_B {'Median': 23.141520928258718}
parallax_faint_K {'Median': 35.949074279472768}
properMotionfaint_K {'Median': 30.111151359989581}
parallax_bright_B {'Median': 0.92410426523842293}
properMotionbright_B {'Median': 0.77791633783101355}
parallax_bright_K {'Median': 0.88986200090022116}
properMotionbright_K {'Median': 0.75686566854465209}
